In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18160327
paper_name = 'dilda_hogg_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sheets = ['Sensitivity','Resistance']

In [11]:
original_data_list = []
for s in sheets:
    original_data = pd.read_excel('raw_data/Tables12.xlsx', sheet_name=s, header=None)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['gene'] = original_data[0].astype(str)
    original_data['gene'] = clean_genename(original_data['gene'])
    original_data.loc[original_data['gene']=='CRS5','gene'] = 'YOR031W' 
    original_data['orf'] = translate_sc(original_data['gene'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data = original_data.loc[t,]
    original_data['data'] = pd.to_numeric(original_data[1], errors='coerce')
    original_data.loc[original_data['data'].isnull()] = 4000
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 384 x 3
       0    1     2
0  PAC10   60  18.3
1  CNM67  204   5.4
2   GIM5  300   3.7
3   CIN1  370   3.0
4   GIM4  372   3.0
                 0    1    2   gene    orf
index_input                               
270          CWH33  807  1.4  CWH33  CWH33
(344, 1)
Original data dimensions: 73 x 3
        0     1    2
0   RPS4B  1950  1.5
1  RPS19A  1808  1.4
2  RPS18B  1805  1.4
3   RPS0B  1743  1.3
4  RPS24B  1625  1.2
Empty DataFrame
Columns: [0, 1, 2, gene, orf]
Index: []
(72, 1)


In [15]:
original_data = pd.concat(original_data_list, axis=0)

In [16]:
original_data.shape

(416, 1)

In [18]:
original_data['data'] = np.log(original_data['data'].values / 1100)

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(415, 1)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [1323]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,1323
data_type,value
orf,
4000,1.290984
YAL011W,-1.203973
YAL012W,-1.804568
YAL013W,-0.749237
YAL021C,-0.839751


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 2


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1323
,data_type,value
gene_id,orf,
9,YAL011W,-1.203973
10,YAL012W,-1.804568
11,YAL013W,-0.749237
19,YAL021C,-0.839751
22,YAL024C,-1.084172


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id           1323          
data_type           value    valuez
gene_id orf                        
9       YAL011W -1.203973 -4.310529
10      YAL012W -1.804568 -6.492379
11      YAL013W -0.749237 -2.658559
19      YAL021C -0.839751 -2.987379
22      YAL024C -1.084172 -3.875314

# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
from IO.save_data_to_db3 import *

In [33]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18160327...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  1.16it/s]

Updating the data_modified_on field...
